2023/10/27 M.Udagawa

# セットアップ

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

n_site = 8
n_boson = n_site
n_sample = 1024
U = 1
t = 1

EPOCH = 101
burn_in = 16

In [ ]:
print(device)

# ネットワークの構築

In [ ]:
class Netwark(nn.Module):
  def __init__(self):
    super(Netwark, self).__init__()
    self.fc_in = nn.Linear(n_site, 80, bias=False)
    self.fc_out = nn.Linear(80, 1, bias=False)

  def forward(self, x:torch.tensor) -> torch.tensor:
    output = self.fc_in(x)
    output = F.tanh(output)
    output = self.fc_out(output)
    return output

  def forward_cpu(self, x):
    output = torch.from_numpy(x.astype(np.float32)).to(device)
    with torch.no_grad():
      output = self.forward(output)
    return output.to('cpu').detach().numpy()

# サンプル生成

In [ ]:
class SampledState():

  def __init__(self, net):
    self.num = np.zeros((n_sample, n_site), dtype=np.int32)
    for i in range(n_sample):
      for j in range(n_boson):
        self.num[i][j%n_site] += 1
    self.thermalize(net)

  def try_flip(self, net):
    num_tmp = np.copy(self.num)
    for i in range(n_sample):
      x0 = np.random.randint(n_site)
      x1 = np.random.randint(n_site)
      if num_tmp[i][x0] > 0 and x0 != x1:
        num_tmp[i][x0] -= 1
        num_tmp[i][x1] += 1

    lnpsi_org = net.forward_cpu(self.num).reshape(n_sample)
    lnpsi_tmp = net.forward_cpu(num_tmp).reshape(n_sample)
    r = np.random.rand(n_sample) #0~1の乱数を出力する
    isflip = r < np.exp(2 * (lnpsi_tmp - lnpsi_org))
    for i in range(n_sample):
      if isflip[i]:
        self.num[i] = num_tmp[i]

  def thermalize(self, net):
    for _ in range(1024):
      self.try_flip(net)


# エネルギー計算

In [ ]:
def LocalEnergy(net, state):
  st = np.zeros((n_sample, n_site, 2, n_site))
  st += state.num.reshape(n_sample, 1, 1, n_site)
  for i in range(n_sample):
    for j in range(n_site):
      if state.num[i][j] > 0:
        st[i][j][0][j] -= 1
        st[i][j][0][(j+1)%n_site] += 1
        st[i][j][1][j] -= 1
        st[i][j][1][(j-1+n_site)%n_site] += 1
  st = st.reshape(n_sample*n_site*2, n_site)
  lnpsi2 = net.forward_cpu(st).reshape(n_sample, n_site, 2)
  lnpsi_org = net.forward_cpu(state.num).reshape(n_sample)
  eloc = np.zeros(n_sample, dtype=np.float32)
  for i in range(n_sample):
    onsite = 0.0
    hopping = 0.0
    for j in range(n_site):
      if state.num[i][j] > 0:
        onsite += state.num[i][j] * (state.num[i][j] - 1)
        hopping += np.sqrt(state.num[i][j]
                           * (state.num[i][(j+1)%n_site] + 1)) \
                           * np.exp(lnpsi2[i][j][0] - lnpsi_org[i])
        hopping += np.sqrt(state.num[i][j]
                           * (state.num[i][(j-1+n_site)%n_site] + 1)) \
                           * np.exp(lnpsi2[i][j][1] - lnpsi_org[i])
    eloc[i] = -t * hopping + 0.5 * U * onsite
  return eloc

# 学習

In [ ]:
net = Netwark()
net = net.to(device)
state = SampledState(net)
optimizer = torch.optim.Adam(net.parameters())
counter = 0

def run(net, state):
  eloc = LocalEnergy(net, state)
  ene = eloc.mean() #mean -> 平均をとる
  eloc = torch.from_numpy(eloc).to(device)
  inputs = torch.from_numpy(state.num.astype(np.float32)).to(device)
  optimizer.zero_grad()
  outputs = net(inputs)
  outputs = torch.reshape(outputs, shape=(n_sample,))
  loss = (outputs * (eloc - ene)).mean()
  loss.backward()
  optimizer.step()
  return ene

In [ ]:
inner_counter = 0

while (inner_counter%EPOCH) != (EPOCH - 1):
  ene = run(net, state)
  for _ in range(burn_in):
    state.try_flip(net)
  if (counter%10) == 0:
    print(counter, ene)
  inner_counter += 1
  counter += 1

In [ ]:
print(state.num)